In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import datetime, itertools
import community

In [2]:
co_offending_table = pd.read_csv('./Cooffending.csv')

# Remove duplicate rows
co_offending_table.drop_duplicates(inplace=True)

# Format the date column as a python datetime object
co_offending_table['Date'] = co_offending_table.Date.apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))

In [ ]:
# dual network: simple, unweighted
dual_network_representation_simple = nx.Graph()

def add_edges_dual_simple(network_representation, crime_event_df):
    #print(crime_event_df.index)
    for (index_1, index_2) in itertools.combinations(crime_event_df.index, 2):
        seq1 = crime_event_df.get_value(index=index_1, col='SeqE')
        seq2 = crime_event_df.get_value(index=index_2, col='SeqE')
        # only will add nodes once 
        #network_representation.add_node(seq1, 
        #                                ncd1=crime_event_df.get_value(index=index_1, col='NCD1'),
        #                                mun=crime_event_df.get_value(index_1, col='MUN'),
        #                                ed1 = crime_event_df.get_value(index_1, col='ED1'))
        #network_representation.add_node(seq2, 
        #                               ncd1=crime_event_df.get_value(index=index_2, col='NCD1'),
        #                                mun=crime_event_df.get_value(index_2, col='MUN'),
        #                                ed1 = crime_event_df.get_value(index_2, col='ED1'))
        # add the edge. Note that this will only ever add one undirected edge
        network_representation.add_edge(seq1, seq2)
    return None

co_offending_table.groupby('NoUnique').apply(lambda x: add_edges_dual_simple(dual_network_representation_simple, x))

In [ ]:
# dual network: mutligraph, unweighted
dual_network_representation_multigraph = nx.MultiGraph()

def add_edges_dual_multi(network_representation, crime_events):
    for (crime_event1, crime_event2) in itertools.combinations(crime_events, 2):
            # in a simple, undirected graph, only one edge can be added 
            network_representation.add_edge(crime_event1, crime_event2)
    return None

co_offending_table.groupby('NoUnique').apply(lambda x: add_edges_dual_multi(dual_network_representation_simple, list(x.SeqE)))

In [ ]:
# Create a bipartite representation of the network
bipartite_network_representation = nx.Graph()

for (index, row) in co_offending_table.iterrows():
    crime_event = 'X_'+str(row['SeqE'])
    offender = row['NoUnique']
    bipartite_network_representation.add_edge(offender, crime_event)

In [ ]:
bipartite_network_representation.nodes()

In [ ]:
largest_component = max(nx.connected_component_subgraphs(bipartite_network_representation), key=len)

In [ ]:
len(largest_component.nodes())

In [ ]:
from networkx.algorithms import bipartite

In [ ]:
bipartite.sets(bipartite_network_representation)